In [1]:
import dgl
import copy
import glob
import math
import torch
import torch.nn as nn
import numpy as np
from os import path
from pathlib import Path
from dgl.data import DGLDataset
from TrainResults import TrainResults
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from SimpleGCN_OnlyNodeFeatures import GCN_OnlyNodeFeatures
from ToyDGLDataset import ToyDGLDataset, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

Using backend: pytorch


In [2]:
# The evaluation function
@torch.no_grad()
def eval(model, device, dataloader):
    model = model.to(device)
    model.eval()
    y_true = []
    y_logits = []

    for batched_graph, labels in dataloader:
        batched_graph = batched_graph.to(device)
        labels = labels.to(device)
        nodeFeatVec = GetNodeFeatureVectors(batched_graph)

        with torch.no_grad():
            pred = model(batched_graph, nodeFeatVec)

        y_true.append(labels.detach().cpu())
        y_logits.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_logits = torch.cat(y_logits, dim = 0)
    y_softmax = nn.functional.softmax(y_logits, dim=1)
    y_scoreClass1 = y_softmax[:, 1]
    y_pred = y_logits.numpy().argmax(1)
    
    num_correct_pred = (y_pred == y_true).sum().item()
    num_total_pred = len(y_true)
    acc =  num_correct_pred / num_total_pred
    
    evalDict = {
        "y_true": y_true.tolist(), 
        "y_logits": y_logits.tolist(), 
        "y_scoreClass1": y_scoreClass1.tolist(),
        "y_pred": y_pred.tolist(), 
        "acc": acc
    }

    return evalDict

In [3]:
def train(model, device, dataloader, optimizer, loss_fn, batchsize, results):
    model = model.to(device)
    model.train()
    
    epochLoss = 0.0
    batchIter = 0
    
    for batched_graph, labels in dataloader:
        batched_graph = batched_graph.to(device)
        labels = labels.to(device)
        nodeFeatVec = GetNodeFeatureVectors(batched_graph)

        #forward
        pred =  model(batched_graph, nodeFeatVec)

        # compute loss
        loss = loss_fn(pred, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # multiply running loss by the number of graphs, 
        # since CrossEntropy loss calculates mean of the losses of the graphs in the batch
        runningTotalLoss = loss.item() #* batchsize
        results.addRunningLoss(runningTotalLoss)
        epochLoss += runningTotalLoss
        batchIter += 1
        
    return epochLoss/batchIter
    

In [4]:
# training
def trainEpochs(model, device, dataloader, optimizer, loss_fn, batchsize, nEpochs):
    results = TrainResults()
    bestModel = None
    bestValAcc = 0.0

    for epoch in range(nEpochs):
        # train
        epochLoss = train(model, device, dataloader, optimizer, loss_fn, batchsize, results)

        # evaluate
        train_result = eval(model, device, train_dataloader)
        val_result = eval(model, device, val_dataloader)
        test_result = eval(model, device, test_dataloader)

        results.addEpochResult(epochLoss, train_result, val_result, test_result)

        if(epoch % 5 == 0):
            results.printLastResult()

        if results.best_val_acc > bestValAcc:
            bestValAcc = results.best_val_acc
            bestModel = copy.deepcopy(model)

    return results, bestModel

In [5]:
def getAllDatasetNames(datasetRootDir):
    files = glob.glob(datasetRootDir + '/*/*/*.json', recursive=True)
    files.sort()
    datasetDirectories = [path.dirname(file) for file in files]
    datasetnames = [path.normpath(dir).split(path.sep)[-1] for dir in datasetDirectories]
    return datasetDirectories, datasetnames

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'

datasetRootDir = '/home/andrew/GNN_Sandbox/GraphToyDatasets_v3'
datasetDirs, datasetNames = getAllDatasetNames(datasetRootDir)

In [7]:
import time
now = time.time()

batchSize = 1024
print(f'Device: {device}')

for i in range(len(datasetDirs)):
    dataset = ToyDGLDataset(datasetNames[i], datasetDirs[i])
    splitIndices = dataset.get_split_indices()

    train_sampler = SubsetRandomSampler(splitIndices['train'])
    val_sampler = SubsetRandomSampler(splitIndices['valid'])
    test_sampler = SubsetRandomSampler(splitIndices['test'])

    train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
    val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
    test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)
    
    # Create the model with given dimensions
    model = GCN_OnlyNodeFeatures(dataset.dim_nfeats, 16, dataset.num_graph_classes).to(device)
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    loss_fn = nn.CrossEntropyLoss()
    epochs = 30
    
    # train
    print(f'Beginning training on dataset {datasetNames[i]}')
    results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
    results.printBestResult()
    
    # save results
    outputFolder = path.join(datasetDirs[i], 'GCN_OnlyNodeFeatures_lossPerTrainingstep__GPU')
    Path(outputFolder).mkdir(parents=True, exist_ok=True)
    
    results.savePlots(outputFolder)
    results.dumpSummary(outputFolder)
    results.pickledump(outputFolder)
    
    # save the best model for inference. (when loading for inference -> model.eval()!! )
    # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
    torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))
    
    print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')

Device: cuda
Done loading data from cached files.
Beginning training on dataset Toy2_v3_0percentDiff
Epoch: 0, Loss: 2.1576, Train: 0.499, Valid: 0.501, Test: 0.496, AUC: 0.501
Epoch: 5, Loss: 0.6978, Train: 0.499, Valid: 0.503, Test: 0.506, AUC: 0.501
Epoch: 10, Loss: 0.6952, Train: 0.499, Valid: 0.505, Test: 0.500, AUC: 0.501
Epoch: 15, Loss: 0.6942, Train: 0.499, Valid: 0.502, Test: 0.503, AUC: 0.501
Epoch: 20, Loss: 0.6938, Train: 0.500, Valid: 0.498, Test: 0.505, AUC: 0.501
Epoch: 25, Loss: 0.6944, Train: 0.502, Valid: 0.497, Test: 0.494, AUC: 0.501
Best epoch: 
Epoch: 13, Loss: 0.6947, Train: 0.497, Valid: 0.508, Test: 0.499, AUC: 0.501
------------------(1/9) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy2_v3_10percentDiff
Epoch: 0, Loss: 0.8140, Train: 0.436, Valid: 0.443, Test: 0.429, AUC: 0.403
Epoch: 5, Loss: 0.6040, Train: 0.700, Valid: 0.695, Test: 0.704, AUC: 0.775
Epoch: 10, Loss: 0.5140, Train: 0.781, Valid: 0.77